In [1]:
# Import the required libraries
import pandas as pd
import re

In [2]:
# Create pandas dataframe and query the dataset
df = pd.read_csv('scraped_data.csv', header=None)
labels = ['title', 'price', 'location', 'bathrooms', 'bedrooms', 'size_msq']

rentals_df = pd.DataFrame(df.values, columns=labels)
rentals_df.head()

,title,price,location,bathrooms,bedrooms,size_msq
0,4 Bedroom Apartment / Flat,KSh 350 000,"Riverside Dr Nairobi, Riverside, Nairobi",4.0,4.0,NaN
1,2 Bedroom Apartment / Flat,KSh 150 000,"Parklands Nairobi, Parklands, Nairobi",2.5,2.0,NaN
2,Commercial Property,KSh 315 000,"Raphta Rd Muthangari, Westlands, Nairobi",NaN,NaN,420 m²
3,Apartment / Flat,KSh 26 000,"Kileleshwa, Kileleshwa, Nairobi",NaN,NaN,NaN
4,1 Bedroom Apartment / Flat,KSh 180 000,"Kilimani, Nairobi",1.0,1.0,NaN


In [3]:
# Data exploration
rentals_df.shape

(242403, 6)

In [4]:
rentals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242403 entries, 0 to 242402
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   title      242403 non-null  object
 1   price      242403 non-null  object
 2   location   217439 non-null  object
 3   bathrooms  225826 non-null  object
 4   bedrooms   227241 non-null  object
 5   size_msq   74025 non-null   object
dtypes: object(6)
memory usage: 11.1+ MB


In [5]:
# Get the number of null records
rentals_df.isna().sum()

title             0
price             0
location      24964
bathrooms     16577
bedrooms      15162
size_msq     168378
dtype: int64

In [6]:
# Drop the 'size_msq' column and create a copy of the dataframe
rentals_df_copy = rentals_df.drop(columns=['size_msq'])
rentals_df_copy.head(3)

,title,price,location,bathrooms,bedrooms
0,4 Bedroom Apartment / Flat,KSh 350 000,"Riverside Dr Nairobi, Riverside, Nairobi",4.0,4.0
1,2 Bedroom Apartment / Flat,KSh 150 000,"Parklands Nairobi, Parklands, Nairobi",2.5,2.0
2,Commercial Property,KSh 315 000,"Raphta Rd Muthangari, Westlands, Nairobi",NaN,NaN


In [7]:
rentals_df_copy.isna().sum()

title            0
price            0
location     24964
bathrooms    16577
bedrooms     15162
dtype: int64

In [8]:
# Separate rows with nulls from those without null values
rows_with_nulls = rentals_df_copy[rentals_df_copy.isna().any(axis=1)]
rows_without_nulls = rentals_df_copy[~rentals_df_copy.isna().any(axis=1)]

rows_without_nulls.isna().sum()

title        0
price        0
location     0
bathrooms    0
bedrooms     0
dtype: int64

In [9]:
# Clean the location column to remove 'Nairobi'
def drop_nairobi(location):
    location.capitalize()
    return location.replace(', Nairobi','')

rows_without_nulls.loc[:,'location'] = rows_without_nulls['location'].apply(drop_nairobi)
rows_without_nulls.head(5)

,title,price,location,bathrooms,bedrooms
0,4 Bedroom Apartment / Flat,KSh 350 000,"Riverside Dr Nairobi, Riverside",4.0,4.0
1,2 Bedroom Apartment / Flat,KSh 150 000,"Parklands Nairobi, Parklands",2.5,2.0
4,1 Bedroom Apartment / Flat,KSh 180 000,Kilimani,1.0,1.0
5,1 Bedroom Apartment / Flat,KSh 28 000,South C,1.0,1.0
6,5 Bedroom House,KSh 70 000,"Quarry Road, Viraj Area, Syokimau",5.0,5.0


In [10]:
# Clean the location column to drop addresses from the location
def clean_location(location):
    # Define the regex pattern
    pattern = re.compile(r'\b[A-Z0-9]{4,6}\+[A-Z0-9]{2,6}\b')
    # Aplply the pattern to remove matching strings and strip extra spaces
    return pattern.sub('', location).strip()

rows_without_nulls.loc[:,'location'] = rows_without_nulls['location'].apply(clean_location)
rows_without_nulls.head(5)

,title,price,location,bathrooms,bedrooms
0,4 Bedroom Apartment / Flat,KSh 350 000,"Riverside Dr Nairobi, Riverside",4.0,4.0
1,2 Bedroom Apartment / Flat,KSh 150 000,"Parklands Nairobi, Parklands",2.5,2.0
4,1 Bedroom Apartment / Flat,KSh 180 000,Kilimani,1.0,1.0
5,1 Bedroom Apartment / Flat,KSh 28 000,South C,1.0,1.0
6,5 Bedroom House,KSh 70 000,"Quarry Road, Viraj Area, Syokimau",5.0,5.0


In [11]:
# Remove the remaining entries with + in them
rows_without_nulls = rows_without_nulls[~rows_without_nulls['location'].str.contains('\+', case=False, na=False)]
rows_without_nulls.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201570 entries, 0 to 242402
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   title      201570 non-null  object
 1   price      201570 non-null  object
 2   location   201570 non-null  object
 3   bathrooms  201570 non-null  object
 4   bedrooms   201570 non-null  object
dtypes: object(5)
memory usage: 9.2+ MB


In [12]:
# Remove entries with https:// in their location
rows_without_nulls = rows_without_nulls[~rows_without_nulls['location'].str.contains('https', case=False, na=False)]
rows_without_nulls.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201552 entries, 0 to 242402
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   title      201552 non-null  object
 1   price      201552 non-null  object
 2   location   201552 non-null  object
 3   bathrooms  201552 non-null  object
 4   bedrooms   201552 non-null  object
dtypes: object(5)
memory usage: 9.2+ MB


In [14]:
# Convert the bedroom and bathroom columns to integers
rows_without_nulls['bathrooms'] = rows_without_nulls['bathrooms'].astype(int)
rows_without_nulls['bedrooms'] = rows_without_nulls['bedrooms'].astype(int)

rows_without_nulls.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201552 entries, 0 to 242402
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   title      201552 non-null  object
 1   price      201552 non-null  object
 2   location   201552 non-null  object
 3   bathrooms  201552 non-null  int64 
 4   bedrooms   201552 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 9.2+ MB


In [15]:
# Remove records with 'POA' i.e. price on asking
rows_without_nulls = rows_without_nulls[~rows_without_nulls['price'].str.contains('POA', case=False, na=False)]
rows_without_nulls.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201472 entries, 0 to 242402
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   title      201472 non-null  object
 1   price      201472 non-null  object
 2   location   201472 non-null  object
 3   bathrooms  201472 non-null  int64 
 4   bedrooms   201472 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 9.2+ MB


In [16]:
# Clean the price column
def clean_price(price):
    clean_price = price.replace('KSh', '').replace(',', '').replace(' ', '').strip()
    # Check if the record contains 'perday'
    if 'perday' in clean_price:
        numeric_part = re.findall(r'\d+', clean_price)
        if numeric_part:
            numeric_value = int(numeric_part[0])
            return numeric_value * 30 # Multiply by 30 if 'perday' is found
        else:
            return None
    elif 'perweek' in clean_price:
        numeric_part = re.findall(r'\d+', clean_price)
        if numeric_part:
            numeric_value = int(numeric_part[0])
            return numeric_value * 4 # Multiply by 4 if 'perweek' is found
        else:
            return None
    else:
        return clean_price

rows_without_nulls.loc[:,'price'] = rows_without_nulls['price'].apply(clean_price)
rows_without_nulls.head(5)

,title,price,location,bathrooms,bedrooms
0,4 Bedroom Apartment / Flat,350000,"Riverside Dr Nairobi, Riverside",4,4
1,2 Bedroom Apartment / Flat,150000,"Parklands Nairobi, Parklands",2,2
4,1 Bedroom Apartment / Flat,180000,Kilimani,1,1
5,1 Bedroom Apartment / Flat,28000,South C,1,1
6,5 Bedroom House,70000,"Quarry Road, Viraj Area, Syokimau",5,5


In [17]:
rows_without_nulls['price'].str.contains('peryear').sum()

3

In [18]:
# Remove records with 'perm²' i.e. per meter squared & 'per year'
rows_without_nulls = rows_without_nulls[~rows_without_nulls['price'].str.contains('perm²', case=False, na=False)]
rows_without_nulls = rows_without_nulls[~rows_without_nulls['price'].str.contains('peryear', case=False, na=False)]
rows_without_nulls.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201462 entries, 0 to 242402
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   title      201462 non-null  object
 1   price      201462 non-null  object
 2   location   201462 non-null  object
 3   bathrooms  201462 non-null  int64 
 4   bedrooms   201462 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 9.2+ MB


In [19]:
# Convert the price column into integer
rows_without_nulls['price'] = rows_without_nulls['price'].astype(int)
rows_without_nulls.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201462 entries, 0 to 242402
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   title      201462 non-null  object
 1   price      201462 non-null  int64 
 2   location   201462 non-null  object
 3   bathrooms  201462 non-null  int64 
 4   bedrooms   201462 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 9.2+ MB


In [20]:
# Clean the titles column
def clean_title(title):
    return title.replace('/ ','').strip()

rows_without_nulls.loc[:,'title'] = rows_without_nulls['title'].apply(clean_title)
rows_without_nulls.head(5)

,title,price,location,bathrooms,bedrooms
0,4 Bedroom Apartment Flat,350000,"Riverside Dr Nairobi, Riverside",4,4
1,2 Bedroom Apartment Flat,150000,"Parklands Nairobi, Parklands",2,2
4,1 Bedroom Apartment Flat,180000,Kilimani,1,1
5,1 Bedroom Apartment Flat,28000,South C,1,1
6,5 Bedroom House,70000,"Quarry Road, Viraj Area, Syokimau",5,5


In [21]:
# Reset the index of the dataframes
rows_without_nulls.reset_index(drop=True, inplace=True)


In [22]:
# Drop any duplicates in the dataset
final_rental_data = rows_without_nulls.drop_duplicates()
final_rental_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70235 entries, 0 to 201460
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      70235 non-null  object
 1   price      70235 non-null  int64 
 2   location   70235 non-null  object
 3   bathrooms  70235 non-null  int64 
 4   bedrooms   70235 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 3.2+ MB


In [24]:
# Write the rental dataframe into a csv file
final_rental_data.to_csv('data/final_rental_records.csv', index=False)
print("CSV file saved successfully.")

CSV file saved successfully.
